# Creating a Sentiment Analysis Web App
### Pytorch and AWS SageMaker
_SageMaker, Lambda, API, CloudWatch_

---
Put an overview of the notebook here

## Outline
1. [Download the data](#download)
2. [Process and prepare the data](#process)
3. [Upload data to S3](#upload)
4. [Build and train the Pytorch model](#train)
5. [Test the trained model](#test)
6. [Deploy the trained model](#deploy)
7. [Use the deployed model for inference](#use)


<a id='download'></a>
## Download the Data

The notebook and model use the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2021-12-23 16:36:13--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  60.3MB/s    in 1.3s    

2021-12-23 16:36:14 (60.3 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



<a id='process'></a>
## Process and Prepare the Data

---
### Read in Data

In [4]:
# necessary imports 
import os
import glob

In [5]:
def read_imbd_data(data_dir='../data/aclImdb'):
    """ Read in IMDb data from aclImdb folder. Creates data and label dictionaries.
    
        Arguments:
        - data_dir: (str) Directory of the data
        
        Returns:
        - data: (dict) Movie review
        - labels: (dict) Movie review labels
    """
    data = {}
    labels = {}
    
    # create paths to read in review data
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            # join path names
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            # open each review and label. Append to dictionaries and label with binary vars
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                       "{}: data size does not equal {}: label size.".format(data_type, sentiment)
                    
    return data, labels

In [6]:
# read in data and display length of train and test data
data, labels = read_imbd_data()
print("IMDb Reviews: Train = {} pos / {} neg <--> Test = {} pos / {} neg".format(len(data['train']['pos']),
                                                                                    len(data['train']['neg']),
                                                                                    len(labels['test']['pos']),
                                                                                    len(labels['test']['pos'])))

IMDb Reviews: Train = 12500 pos / 12500 neg <--> Test = 12500 pos / 12500 neg


In [7]:
data['train']['pos'][0]

'My family goes back to New Orleans late 1600\'s early 1700\'s and in watching the movie I knew it was a history my grand-parents never talked about, but we knew it existed. I have cousins obviously black aka African Americans and others who can "pass" as white and chose not to. It\'s a hard history to watch when you realize that it\'s your family they\'re talking about and that Cane River is all a part of that history. It makes me want to cry and it makes me want to kick the \'arse\' of my great grandfathers who owned those plantations and wonder in awe of how my great grandmothers of African heritage lived under that oppressive and yet aristocratic existence...And at the same time had I not come out of that history, I probably wouldn\'t be the successful business woman I am today living successfully in a fairly integrated world. The acting was both excellent and fair depending upon the actor, but it is a movie that NEEDED to be made. Anne Rice is incredible and I ask myself, why is s

In [8]:
labels['train']['pos'][0]

1

---
### Create Feature and Target Sets
Combine the training and test data/labels and shuffle to creat feature and target sets.

In [9]:
from sklearn.utils import shuffle

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re
from bs4 import BeautifulSoup

import pickle

In [10]:
def combine_imdb_data(data, labels):
    """ Combine pos and neg reviews from the training and test data.
        
        Arguments:
        - data: (dict) Unprocessed reviews
        - labels: (dict) Sentiment label, 1 pos --> 0 neg
        
        Returns:
        - train_X, test_X: features
        - train_y, test_y: targets
    """
    # combine positive and negative reviews and labels
    train_data = data['train']['pos'] + data['train']['neg']
    test_data = data['test']['pos'] + data['test']['neg']
    train_labels = labels['train']['pos'] + labels['train']['neg']
    test_labels = labels['test']['pos'] + labels['test']['neg']
    
    # shuffle reviews and labels within training and test sets
    train_data, train_labels = shuffle(train_data, train_labels)
    test_data, test_labels = shuffle(test_data, test_labels)
    
    return train_data, test_data, train_labels, test_labels

In [11]:
train_X, test_X, train_y, test_y = combine_imdb_data(data, labels)
print("IMDb Data Length: Train data = {}, Test data = {}".format(len(train_X), len(test_X)))

IMDb Data Length: Train data = 25000, Test data = 25000


In [12]:
# take a look at a review and it's corresponding label
print(train_X[20], '\n')
print(train_y[20])

Hopefully the score has changed by now due to my brilliant and stunning review which persuades all of you to go and watch the film thereby creating an instant chorus of "8"s, this movie's true score.<br /><br />As mentioned before Chris Rock is The King! Previous to going to see this movie I wasn't that over the top about him but now I'm banging on the doors of Chris Rock's website begging him to take me on as his protege. This film is truly funny, if you don't find this movie funny you REALLY need therapy and it's humour which targets all areas of society including race(predictably), class division, love, wealth, employment, dreams, stand up comedy... the list goes on.<br /><br />There was one slight disappointment for me however. This was that in going into this film I didn't realise that it was actually a remake of "Heaven Can Wait" another quite good movie made in 1971 with Warren Beatty. As such I was quite surprised when I watched this movie and suddenly the plot began to unravel

---
### Process Review
Remove the html formatting and convert the review into a list of words.

In [13]:
def review_to_words(review):
    """ Converts a review string to a list of words. Removes html
        formatting, stopwords and morphological endings of common
        words.
        
        Arguments:
        - review: (str) String of words that make up review
        
        Returns:
        - words: (list) List of processed words in a review
    
    """ 
    nltk.download('stopwords', quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, 'html.parser').get_text() # remove html tags
    text = re.sub(r"[^a-zA-z0-9]", " ", text.lower()) 
    words = text.split() # split the string into a list of words
    words = [word for word in words if word not in stopwords.words('english')] # remove stopwords
    words = [stemmer.stem(word) for word in words] # stem words
    
    return words

In [14]:
words = review_to_words(train_X[20])
print(words)

['hope', 'score', 'chang', 'due', 'brilliant', 'stun', 'review', 'persuad', 'go', 'watch', 'film', 'therebi', 'creat', 'instant', 'choru', '8', 'movi', 'true', 'score', 'mention', 'chri', 'rock', 'king', 'previou', 'go', 'see', 'movi', 'top', 'bang', 'door', 'chri', 'rock', 'websit', 'beg', 'take', 'proteg', 'film', 'truli', 'funni', 'find', 'movi', 'funni', 'realli', 'need', 'therapi', 'humour', 'target', 'area', 'societi', 'includ', 'race', 'predict', 'class', 'divis', 'love', 'wealth', 'employ', 'dream', 'stand', 'comedi', 'list', 'goe', 'one', 'slight', 'disappoint', 'howev', 'go', 'film', 'realis', 'actual', 'remak', 'heaven', 'wait', 'anoth', 'quit', 'good', 'movi', 'made', '1971', 'warren', 'beatti', 'quit', 'surpris', 'watch', 'movi', 'suddenli', 'plot', 'began', 'unravel', 'distinctli', 'similar', 'older', 'movi', 'watch', 'tv', 'week', 'ago', 'regardless', 'movi', 'opinion', 'better', 'version', 'two', 'simpli', 'differ', 'area', 'cover', 'fact', 'chri', 'rock', 'funnier', 'w

In [15]:
cache_dir = os.path.join("../cache", "sentiment_analysis")
os.makedirs(cache_dir, exist_ok=True) 

def preprocess_data(train_data, test_data, train_labels, test_labels,
                    cache_dir=cache_dir, cache_file='preprocesssed_data.pkl'):
    """ Convert each review to words and read from the cache file if 
        available. 
    
    """
    # if cache file exists try to read from it
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f: # open and read binary file
                cache_data = pickle.load(f)
                print("Reading preprocessed data from cache file: {}".format(cache_file))
        except:
            pass
        
    # if cache data does not exist create it
    if cache_data is None:
        # process data to create list of words for each review
        train_words = [review_to_words(review) for review in train_data]
        test_words = [review_to_words(review) for review in test_data]
        
        # write to cache file if it doesn't exist
        if cache_file is not None:
            cache_data = dict(train_words=train_words, test_words=test_words,
                              train_labels=train_labels, test_labels=test_labels)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file: {}".format(cache_file))
            
    else:
        # unpack data from cache file
        train_words = cache_data['train_words']
        test_words = cache_data['test_words']
        train_labels = cache_data['train_labels']
        test_labels = cache_data['test_labels']
        
    return train_words, test_words, train_labels, test_labels

In [16]:
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocesssed_data.pkl


---
### Transform the Data
First we will create a working vocabulary of the most frequently occuring words in our dataset. We will remove the words that occur most infrequently. Each review will be fixed in size with shorter reviews padded with zeros. This will allow our RNN to train more efficiently.

In [17]:
# necessary imports
import numpy as np
from collections import Counter

In [18]:
def build_dict(data, vocab_size=5000):
    """ Construct and return a dictionary mapping each of the most frequently 
        appearing words to a unique integer.
        
        Arguments:
        - data: preprocessed reviews
        - vacab_size: (int) size of vacabulary
        
        Returns:
        - word_dict: dictionary of vocabulary mappings
    """
    # count and sort the words 
    word_count = Counter(np.concatenate(train_X, axis = 0))
    sorted_vocab = sorted(word_count, key=word_count.get, reverse=True)
    
    # create a word dictionary
    word_dict = {word: idx+2 for idx, word in enumerate(sorted_vocab[:vocab_size-2])}
    
    return word_dict

Take a look at the dictionary to make sure everything looks good. 

In [20]:
word_dict = build_dict(train_X)
# print(word_dict)

In [21]:
most_freq = [key for idx, (key, val) in enumerate(word_dict.items()) if idx < 5]
print(most_freq)

['movi', 'film', 'one', 'like', 'time']


### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [22]:
data_dir = '../data/pytorch' # folder that will store the data
if not os.path.exists(data_dir): # check if the folder exists
    os.makedirs(data_dir)

In [23]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the Reviews
Convert the reviews into their integer sequence representation. Shorter reviews will be padded with `0` or `1` for no word and infrequent word representation. Longer reviews will be truncated to 500 characters. 

In [24]:
review_lens = Counter([len(review) for review in train_X])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Max length review: {}".format(max(review_lens)))
print(len(train_X))

Zero-length reviews: 0
Max length review: 1429
25000


In [26]:
def convert_and_pad(word_dict, review, pad=500):
    NOWORD = 0
    INFREQ = 1
    current_review = [NOWORD] * pad
        
    for idx, word in enumerate(review[:pad]):
        if word in word_dict:
            current_review[idx] = word_dict[word]
        else:
            current_review[idx] = INFREQ
    
    return current_review, min(len(review), pad)

def convert_reviews(word_dict, data, pad=500):
    """ Convert each review to an integer sequence representation. Truncate
        to 500 chars and pad with 0s and 1s accordingly.
        
        Arguments:
        - word_dict: (dict) word mapping dictionary
        - data: reviews
        - pad: (int) length to truncate to
        
        Returns:
        - train_X: feature
        - train_X_len: length of feature set
    """    
    result = []
    lengths = []
    
    for review in data:
        current_review, leng = convert_and_pad(word_dict, review, pad)
        result.append(current_review)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [27]:
train_X, train_X_len = convert_reviews(word_dict, train_X)

In [28]:
test_X, test_X_len = convert_reviews(word_dict, test_X)

In [29]:
print(train_X[0], '\n')
print(train_X_len[20])

[1974 3354    1    1   42 3932  618   91    1  219  279  816   37  207
   89  401   11    1    1    1  122 1803 1386  345   89  175   52  237
 2286 1349 1938  170  471 1352 2566   48  271  345  126 2162 2336   50
 1425    1   50    1   89    1   10   31   62 2829 3845   45 3439    5
  451  816  707   29    1  237 2502  875    1  345  116    6   85  296
   14   37 2502   62  603 2393 4211    1 4632   26   85  105 1707 1205
  273 2143  593  400 1733 1087 1899 1160 2829   78 3439 2829   28   35
   45  202   30 1087 1899   33   63   17 4674 3905  658 2336    1  945
  359 1877  352    1 2991 1425  136 1238  321    1  711   44   72 2272
    1  356  193   52 1279   50  128 3932    1    1   26  193   60  794
 3048 3728 3330   33 1948  489 1345    1    1  384 1342 2503   70   26
  465    2  658  593  400  397   19 1205 1899   60  427  219 4815  633
    1    5   37    1   96   83  208  169 1087  593    1    1   52 2272
    1   29  891 3905  593    1    1  785  344    4   48  192  777   92
 1707 

<a id='upload'></a>
## Upload the Data to S3
Save the data locally and upload to S3 later. Note that the format has to be in the form `label`, `length`, `review`.

### Save and process locally
It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of 500 integers representing the words in the review.

In [31]:
# necessary imports
import pandas as pd
import sagemaker

In [32]:
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Upload the data
Create a SageMaker session, role, and bucket. Upload the data to the default S3 bucket.


In [33]:
# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
session = sagemaker.Session()

bucket = session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = sagemaker.get_execution_role()

In [34]:
training_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

In [35]:
print(session.default_bucket())

sagemaker-us-west-2-904606187431


<a id='train'></a>
## Build and Train the Pytorch Model

In [36]:
!pygmentize train/model.py

# PROGRAMMER: Justin Bellucci 
# DATE CREATED: 07_31_2020                                  
# REVISED DATE: 12_23_2021

import torch.nn as nn
import torch

class LSTMClassifier(nn.Module):
    """ LSTM based RNN to perform sentiment analysis
    
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers=2):
        """ Initialize the model by setting up the various 
            layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)
        
        self.fc = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        
        batch_size = x.size(0)
        x = x.t()
        reviews_lengths = x[0,:] # first row is the length of original review
        reviews = x[1:,:] # second row to end is each review
        
        embeds = self.embeddin

Loading in a bit of data to test the model before we use GPU to train on Sagemaker. This is important to identify any mistakes.

In [37]:
# necessary imports
import torch
import torch.utils.data
import torch.nn as nn

import torch.optim as optim


In [38]:
batch_size = 50

# read in the fist 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# turn the Pandas DF into Tensors. Labels are first.
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# build dataset using TensorDataset() from Pytorch
train_sample_dataset = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)

# build the dataloader using DataLoader() from Pytorch
train_sample_loader = torch.utils.data.DataLoader(train_sample_dataset, batch_size=batch_size)

In [39]:
train_sample.transpose()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1,0,1,1,0,1,1,1,1,1,...,1,0,1,0,0,1,0,1,1,1
1,227,84,89,146,60,332,93,330,108,330,...,19,221,164,141,126,43,123,180,69,41
2,1974,6,69,2863,300,634,7,709,1,1,...,16,537,907,311,979,2,55,2513,180,16
3,3354,2,120,506,174,1050,3,217,1133,1505,...,439,1,3,657,2,620,1,34,1,77
4,1,1,48,356,2,2026,6,177,16,68,...,7,915,211,12,340,72,95,6,261,2668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
for idx, (x, y) in enumerate(train_sample_loader):
#     print(x.t()[0,:])
    print(x.t()[1:,:])

tensor([[1974,    6,   69,  ...,  356, 1192,   50],
        [3354,    2,  120,  ...,   46, 1260,   10],
        [   1,    1,   48,  ...,   23,  158, 1715],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[ 303,  135,  138,  ...,    1,    1,  161],
        [  70,    2,  810,  ..., 1464,  172,   53],
        [ 102,  514,  408,  ..., 3358,  597,  269],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[ 487,  135,  581,  ...,  796,   19,  967],
        [   1,    1,    2,  ...,    3,  443, 1950],
        [  12,    3,  497,  ...,  161,  198, 3269],
        ...,
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0]])
tensor([[2494,  815,  

### Training with the small sample dataset

In [66]:
# %load_ext autoreload
%autoreload 2

from train.model import LSTMClassifier

In [67]:
def train_sample(model, train_loader, epochs, optimizer, criterion, device):
    """ Train a sample dataset in Jupyter notebook.
    """
    for e in range(epochs):
        model.train() # put model in training mode
        total_loss = 0
        
        for batch in train_loader:
            batch_X, batch_y = batch
            
            # move to GPU if available
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
#             h = tuple([each.data for each in h])
            
            # train the model
            optimizer.zero_grad() # zero gradients
            out= model.forward(batch_X)
            
            loss = criterion(out, batch_y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(e+1, total_loss/len(train_loader)))

In [68]:
embedding_dim = 32
hidden_dim = 100
vocab_size = 5000
epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} to train...".format(device))

model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

train_sample(model, train_sample_loader, epochs, optimizer, criterion, device)

Using cpu to train...
Epoch: 1, BCELoss: 0.692295241355896
Epoch: 2, BCELoss: 0.6862651824951171
Epoch: 3, BCELoss: 0.6809724569320679
Epoch: 4, BCELoss: 0.6730151653289795
Epoch: 5, BCELoss: 0.6586886048316956


<a id='train'></a>
## Train the Model

In [73]:
# necessary imports
from sagemaker.pytorch import PyTorch

In [74]:
pytorch_estimator = PyTorch(entry_point="train.py",
                            source_dir="train",
                            role=role,
                            framework_version='1.8.0',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p2.xlarge',
                            hyperparameters={'epochs': 15,
                                             'hidden_dim': 300})

In [75]:
pytorch_estimator.fit({'training': training_data})

2021-12-23 18:17:23 Starting - Starting the training job...
2021-12-23 18:17:47 Starting - Launching requested ML instancesProfilerReport-1640283443: InProgress
......
2021-12-23 18:18:47 Starting - Preparing the instances for training.........
2021-12-23 18:20:20 Downloading - Downloading input data...
2021-12-23 18:20:48 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-23 18:24:54,999 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-23 18:24:55,025 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-23 18:24:55,042 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-23 18:24:55,472 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
2

<a id='deploy'></a>
## Deployment

In [88]:
# training_job_name = "pytorch-training-2021-12-23-18-17-23-313"
import boto3
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

model_artifacts = pytorch_estimator.model_data
model_artifacts

boto_session = boto3.session.Session()
region = boto_session.region_name
print("Region: ",region)
print("Role: ",role)
print("s3 Bucket: ",bucket)
print("Session: ",session)
print("S3 Job Prefix: ",prefix)

Region:  us-west-2
Role:  arn:aws:iam::904606187431:role/service-role/AmazonSageMaker-ExecutionRole-20211223T092713
s3 Bucket:  sagemaker-us-west-2-904606187431
Session:  <sagemaker.session.Session object at 0x7ffb95a76a20>
S3 Job Prefix:  sagemaker/sentiment_rnn


In [104]:
image_uri = pytorch_estimator.training_image_uri()
print(image_uri)

763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.8.0-gpu-py3


In [107]:
# from sagemaker.pytorch import PyTorchModel

# pytorch_model = PyTorchModel(model_data=model_artifacts,
#                              role=role,
#                              framework_version='1.8.0',
#                              source_dir='serve',
#                              entry_point='predict.py',
#                              py_version='py3')

pytorch_uri = 'pytorch-inference:1.10.0-cpu-py38-ubuntu20.04-e3'

### Model Creation
Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [108]:
from time import gmtime, strftime

model_name = "pytorch-serverless" + strftime("%Y-%m-a%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": pytorch_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

Model name: pytorch-serverless2021-12-a23-21-27-56


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Using non-ECR image "pytorch-inference:1.10.0-cpu-py38-ubuntu20.04-e3" without Vpc repository access mode is not supported.

### Endpoint Configuration Creation

This is where you can adjust the <b>Serverless Configuration</b> for your endpoint. The current max concurrent invocations for a single endpoint, known as <b>MaxConcurrency</b>, can be any value from <b>1 to 50</b>, and <b>MemorySize</b> can be any of the following: <b>1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB</b>.

In [100]:
pytorch_epc_name = "pytorch-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=pytorch_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint-config/pytorch-serverless-epc2021-12-23-20-58-54


### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [101]:
endpoint_name = "pytorch-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=pytorch_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:904606187431:endpoint/pytorch-serverless-ep2021-12-23-21-00-00


In [285]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [286]:
def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, pytorch_predictor.predict(array))
        
    return predictions

In [287]:
# response = pytorch_predictor.predict(test_X)
# print(response)
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [290]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.5

<a id='use'></a>
## Use the Deployed Model for Inference